В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

# Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.



In [80]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

digits_dataset = datasets.load_digits()
X_train, X_test, y_train, y_test = train_test_split(digits_dataset.data, 
                                                    digits_dataset.target, 
                                                    test_size=0.25,
                                                    shuffle = False)


In [81]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(1347, 64)
(450, 64)
(1347,)
(450,)


In [82]:
from scipy.spatial import distance
import numpy as np

In [83]:
result = []
for test_x in X_test:
    cur_len = None
    cur_index = 0
    for index, train_x in enumerate(X_train):
        dist = distance.euclidean(train_x, test_x)
        if cur_len == None:
            cur_len = dist
        elif dist < cur_len:
            cur_index = index
            cur_len = dist
    result.append(cur_index)

In [84]:
y_result = y_train[result]

In [85]:
def write_answer_nn(answer, number):
    print 'Answer {0}'.format(answer)
    with open("1nn_answer{0}.txt".format(number), "w") as fout:
        fout.write(str(answer))
        
write_answer_nn(float(np.count_nonzero(y_test - y_result)) / y_result.shape[0], 1)

Answer 0.0377777777778


Верно. Обратите внимание - ошибка составляет менее 5%! Это показательный пример, что никогда не стоит забывать о бейзлайнах - иногда легко податься соблазну запустить несколько сложных алгоритмов, получить в них качество 90+%, и верить в то, что задача решена хорошо. При этом же запросто может оказаться, что простые методы дают еще лучший результат.

# Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [86]:
from sklearn.ensemble import RandomForestClassifier

In [87]:
clf = RandomForestClassifier(n_estimators=1000)

In [88]:
clf.fit(X_train, y_train)
prediction = clf.predict(X_test)

In [89]:
write_answer_nn(1 - clf.score(X_test, y_test), 2)

Answer 0.0622222222222


Обратите внимание, что качество заметно хуже, чем у 1NN. Это нестандартная ситуация, как правило Random Forest отрабатывает лучше, но такие ситуации иногда встречаются.